# Time series regression - Store sales forecasting, Part 1

Ahmet Zamanis

## Introduction

Introduce TS regression, dataset and competition, part 1

## Data preparation

When you click the **Render** button a document will be generated that
includes both content and the output of embedded code. You can embed
code like this:

In [1]:
1 + 1

2

You can add options to executable code like this

4

The `echo: false` option disables the printing of code (only output is
displayed).

## Exploratory analysis - Time & calendar effects

## Feature engineering - Time & calendar features

## Model 1 - Time effects decomposition

## Exploratory analysis - Lags & covariates

## Feature engineering - Lags & covariates

## Model 2 - Hybrid model

## Conclusion